In [1]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 853.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import transformers
from sentence_transformers import SentenceTransformer, util

In [3]:
model = SentenceTransformer("ai-forever/ru-en-RoSBERTa")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/241 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.49M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [4]:
raw_data = pd.read_csv('эмоции32.csv', encoding='windows-1251', sep=';')
raw_data.head(8)

,Название эмоции,Коллокаты из скетчей НКРЯ,"Предсказания YandexGPT Pro 5: Я испытываю эмоцию, которая характеризуется следующими контекстными маркерами.",Каким словом можно назвать эту эмоцию?,Я ребенок. Я испытываю Х.,Я взрослый. Я испытываю Х.,Я женщина. Я испытываю Х.,Я мужчина. Я испытываю Х.,Определение из Викисловаря,Контексты из Викисловаря,Unnamed: 10,Ребенок,Взрослый,Женщина,Мужчина,Лепесток
0,Гнев,"Определения\n1. праведный\n10,36\n2. справедли...","На основе представленных контекстных маркеров,...","На основании представленных маркеров, наиболее...",Поняла. Давай разберемся с твоими чувствами. К...,"Понимаю, что вы испытываете сильные эмоции. Во...","Дорогая, понимаю, что сейчас ты испытываешь си...","Поняла, давай поговорим о том, как справиться ...","чувство сильного, яростного возмущения","? Тогда он постарался представить себе, в как...",NaN,1. Сильный.\n2. Вспыльчивый.\n3. Резкий.\n4. Г...,1. Контролируемый.\n2. Обоснованный.\n3. Накоп...,Вспыльчивый.\nРезкий.\nНеудержимый.\nОбиженный...,1. Яростный.\n2. Свирепый.\n3. Буйный.\n4. Неи...,5.0
1,Изумление,"Определения\n1. неподдельный\n8,53\n2. радостн...","Судя по представленным контекстным маркерам, в...",Наиболее точно эту эмоцию описывает слово “вос...,"Ох, как здорово, что ты испытываешь такое удив...","Изумление - действительно удивительная эмоция,...","Дорогая, ваше изумление - это совершенно естес...","Изумление - это мощная эмоция, которая может п...",крайнее удивление,Инспектор внезапно умолк и в крайнем изумлении...,NaN,Большое.\nНеожиданное.\nСильное.\nРадостное.\n...,1. Неподдельное.\n2. Искреннее.\n3. Наивное.\n...,1. Неожиданное.\n2. Сильное.\n3. Радостное.\n4...,1. Неожиданное.\n2. Сильное.\n3. Глубокое.\n4....,8.0
2,Отвращение,"Определения\n1. непреодолимый\n9,86\n2. инстин...","Судя по представленным контекстным маркерам, в...","Исходя из представленных контекстных маркеров,...","Если ты испытываешь отвращение, важно поговори...","Если вы испытываете отвращение, вот что важно ...","Если вы испытываете отвращение, важно разобрат...","Если вы испытываете отвращение, важно понять е...","крайняя неприязнь, в высшей степени отрицатель...","И с отвращением читая жизнь мою, // Я трепещу...",NaN,Горькое.\nЛипкое.\nТошнотворное.\nПротивное.\n...,1. Неприятное.\n2. Тошнотворное.\n3. Мерзкое.\...,1. Тошнотворное.\n2. Мерзкое.\n3. Гадкое.\n4. ...,1. Тошнотворное.\n2. Мерзкое.\n3. Гадкое.\n4. ...,6.0
3,Ужас,"Определения\n1. панический\n9,67\n2. суеверный...","Судя по представленным контекстным маркерам, в...","Исходя из представленных контекстных маркеров,...","Если ты испытываешь ужас, вот что тебе нужно з...","Если вы испытываете ужас, важно действовать ко...","Если вы испытываете ужас, важно предпринять сл...","Если вы испытываете ужас, важно понимать, что ...","чувство иррационального панического страха, до...","Всех присутствующих охватил ужас, до того нео...",NaN,1. Страшный.\n2. Жуткий.\n3. Кошмарный.\n4. Пу...,1. Невыносимый.\n2. Панический.\n3. Беспросвет...,1. Непередаваемый.\n2. Острый.\n3. Гнетущий.\n...,1. Невыносимый.\n2. Гнетущий.\n3. Острый.\n4. ...,1.0
4,Восхищение,"Определения\n1. нескрываемый\n8,5\n2. неподдел...","На основе представленных контекстных маркеров,...",Исходя из представленных маркеров и характерис...,"Когда ребенок испытывает восхищение, это очень...","Как взрослый человек, испытывающий восхищение,...","Как женщина, испытывающая восхищение, вы может...","Как мужчина, испытывающий восхищение, вы может...","действие по значению гл. восхищаться, ощущение...","Она улыбалась, качала головой и тут же очень к...",NaN,1. Восточное.\n2. Волшебное.\n3. Удивительное....,1. Искреннее.\n2. Глубокое.\n3. Безграничное.\...,1. Восторженное.\n2. Искреннее.\n3. Грандиозно...,1. Искреннее.\n2. Грандиозное.\n3. Глубокое.\n...,2.0
5,Восторг,"Определения\n1. неописанный\n9,17\n2. неописуе...","На основе представленных контекстных маркеров,...","На основе представленных контекстных маркеров,...","Как ребенок, испытывающий восторг, ты можешь з...","К

In [5]:
del raw_data['Unnamed: 10']
raw_data['Название эмоции'] = raw_data['Название эмоции'].str.lower()
raw_data.iloc[:, :-1] = raw_data.iloc[:, :-1].map(lambda x: x.replace('\n', ' '))

In [6]:
df = pd.DataFrame()

df['Название эмоции'] = raw_data['Название эмоции']
df['Ребенок'] = raw_data['Ребенок']
df['Взрослый'] = raw_data['Взрослый']
df['Мужчина'] = raw_data['Мужчина']
df['Женщина'] = raw_data['Женщина']

df.head()

,Название эмоции,Ребенок,Взрослый,Мужчина,Женщина
0,гнев,1. Сильный. 2. Вспыльчивый. 3. Резкий. 4. Горя...,1. Контролируемый. 2. Обоснованный. 3. Накопив...,1. Яростный. 2. Свирепый. 3. Буйный. 4. Неисто...,Вспыльчивый. Резкий. Неудержимый. Обиженный. Я...
1,изумление,Большое. Неожиданное. Сильное. Радостное. Удив...,1. Неподдельное. 2. Искреннее. 3. Наивное. 4. ...,1. Неожиданное. 2. Сильное. 3. Глубокое. 4. Шо...,1. Неожиданное. 2. Сильное. 3. Радостное. 4. У...
2,отвращение,Горькое. Липкое. Тошнотворное. Противное. Жутк...,1. Неприятное. 2. Тошнотворное. 3. Мерзкое. 4....,1. Тошнотворное. 2. Мерзкое. 3. Гадкое. 4. Оме...,1. Тошнотворное. 2. Мерзкое. 3. Гадкое. 4. Оме...
3,ужас,1. Страшный. 2. Жуткий. 3. Кошмарный. 4. Пугаю...,1. Невыносимый. 2. Панический. 3. Беспросветны...,1. Невыносимый. 2. Гнетущий. 3. Острый. 4. Бес...,1. Непередаваемый. 2. Острый. 3. Гнетущий. 4. ...
4,восхищение,1. Восточное. 2. Волшебное. 3. Удивительное. 4...,1. Искреннее. 2. Глубокое. 3. Безграничное. 4....,1. Искреннее. 2. Грандиозное. 3. Глубокое. 4. ...,1. Восторженное. 2. Искреннее. 3. Грандиозное....


In [7]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+\.', '', text)
    text = re.sub(r'\.', '', text)
    return text

df['Ребенок'] = df['Ребенок'].apply(clean_text)
df['Взрослый'] = df['Взрослый'].apply(clean_text)
df['Мужчина'] = df['Мужчина'].apply(clean_text)
df['Женщина'] = df['Женщина'].apply(clean_text)

df.head()

,Название эмоции,Ребенок,Взрослый,Мужчина,Женщина
0,гнев,сильный вспыльчивый резкий горячий злой ...,контролируемый обоснованный накопившийся с...,яростный свирепый буйный неистовый гневли...,вспыльчивый резкий неудержимый обиженный ярост...
1,изумление,большое неожиданное сильное радостное удивител...,неподдельное искреннее наивное глубокое и...,неожиданное сильное глубокое шокрующее ош...,неожиданное сильное радостное удивляющее ...
2,отвращение,горькое липкое тошнотворное противное жуткий г...,неприятное тошнотворное мерзкое отталкиваю...,тошнотворное мерзкое гадкое омерзительное ...,тошнотворное мерзкое гадкое омерзительное ...
3,ужас,страшный жуткий кошмарный пугающий мрачны...,невыносимый панический беспросветный катас...,невыносимый гнетущий острый беспросветный ...,непередаваемый острый гнетущий беспомощный...
4,восхищение,восточное волшебное удивительное яркое ск...,искреннее глубокое безграничное настоящее ...,искреннее грандиозное глубокое сильное во...,восторженное искреннее грандиозное умилённ...


In [8]:
df[df.columns[1]][0]

' сильный  вспыльчивый  резкий  горячий  злой  бурный  яростный  сердитый  нетерпеливый  грозовой'

In [9]:
embed = pd.DataFrame()

embed['Название эмоции'] = df['Название эмоции']

for i in range(1, 5):
    row = []
    for j in range(32):
      temp = model.encode(df[df.columns[i]][j])
      row.append(temp)
    embed[df.columns[i]] = row

In [10]:
len(embed.iloc[0, 1])

1024

In [12]:
emotions_names = list(df['Название эмоции'])
personas_names = ['Ребенок', 'Взрослый', 'Мужчина', 'Женщина']

In [13]:
from itertools import combinations

persons = [
    {'person_id': 0, 'person_name': 'Ребенок'},
    {'person_id': 1, 'person_name': 'Взрослый'},
    {'person_id': 2, 'person_name': 'Мужчина'},
    {'person_id': 3, 'person_name': 'Женщина'}
]

emotions = emotions_names

# Создаем DataFrame всех (person, emotion)
data = []
for p in persons:
    for e in emotions:
        data.append({
            'person_id': p['person_id'],
            'person_name': p['person_name'],
            'emotion': e
        })

df = pd.DataFrame(data)

# Создаем все уникальные пары без повторений
pairs = list(combinations(df.index, 2))

# Формируем результат
result_rows = []
for i, j in pairs:
    row1 = df.loc[i]
    row2 = df.loc[j]

    idx1 = emotions.index(row1['emotion'])
    idx2 = emotions.index(row2['emotion'])

    vec1 = embed.iloc[idx1][row1['person_name']]
    vec2 = embed.iloc[idx2][row2['person_name']]

    cos_sim_value = util.cos_sim(vec1, vec2).item()

    is_diff_emotion = 1 if row1['emotion'] != row2['emotion'] else 0
    is_diff_person = 1 if row1['person_id'] != row2['person_id'] else 0

    result_rows.append({
        'person1_id': row1['person_id'],
        'person1_name': row1['person_name'],
        'person2_id': row2['person_id'],
        'person2_name': row2['person_name'],
        'emotion1': row1['emotion'],
        'emotion2': row2['emotion'],
        'cos': cos_sim_value,
        'Is_diff_emotion': is_diff_emotion,
        'Is_diff_person': is_diff_person
    })

result_df = pd.DataFrame(result_rows)

print(result_df.shape)  # (8128, 7)
result_df

(8128, 9)


,person1_id,person1_name,person2_id,person2_name,emotion1,emotion2,cos,Is_diff_emotion,Is_diff_person
0,0,Ребенок,0,Ребенок,гнев,изумление,0.559539,1,0
1,0,Ребенок,0,Ребенок,гнев,отвращение,0.480251,1,0
2,0,Ребенок,0,Ребенок,гнев,ужас,0.712524,1,0
3,0,Ребенок,0,Ребенок,гнев,восхищение,0.615569,1,0
4,0,Ребенок,0,Ребенок,гнев,восторг,0.678856,1,0
...,...,...,...,...,...,...,...,...,...
8123,3,Женщина,3,Женщина,разочарование,презрение,0.743977,1,0
8124,3,Женщина,3,Женщина,разочарование,агрессия,0.556850,1,0
8125,3,Женщина,3,Женщина,раскаяние,презрение,0.578382,1,0
8126,3,Женщина,3,Женщина,раскаяние,агрессия,0.478956,1,0


In [14]:
  def extract_gender(name):
      if name == 'Мужчина':
          return 'Male'
      elif name == 'Женщина':
          return 'Female'
      else:
          return np.nan

  def extract_age(name):
      if name == 'Ребенок':
          return 'Child'
      elif name == 'Взрослый':
          return 'Adult'
      else:
          return np.nan

  test_persons = result_df

  test_persons['person1_gender'] = test_persons['person1_name'].apply(extract_gender)
  test_persons['person2_gender'] = test_persons['person2_name'].apply(extract_gender)

  test_persons['person1_age'] = test_persons['person1_name'].apply(extract_age)
  test_persons['person2_age'] = test_persons['person2_name'].apply(extract_age)

  # Создаем признаки различий
  test_persons['Is_diff_gender'] = np.where(
      test_persons[['person1_gender', 'person2_gender']].notna().all(axis=1),
      test_persons['person1_gender'] != test_persons['person2_gender'],
      np.nan
  )

  test_persons['Is_diff_age_group'] = np.where(
      test_persons[['person1_age', 'person2_age']].notna().all(axis=1),
      test_persons['person1_age'] != test_persons['person2_age'],
      np.nan
  )

  test_persons

,person1_id,person1_name,person2_id,person2_name,emotion1,emotion2,cos,Is_diff_emotion,Is_diff_person,person1_gender,person2_gender,person1_age,person2_age,Is_diff_gender,Is_diff_age_group
0,0,Ребенок,0,Ребенок,гнев,изумление,0.559539,1,0,NaN,NaN,Child,Child,NaN,0.0
1,0,Ребенок,0,Ребенок,гнев,отвращение,0.480251,1,0,NaN,NaN,Child,Child,NaN,0.0
2,0,Ребенок,0,Ребенок,гнев,ужас,0.712524,1,0,NaN,NaN,Child,Child,NaN,0.0
3,0,Ребенок,0,Ребенок,гнев,восхищение,0.615569,1,0,NaN,NaN,Child,Child,NaN,0.0
4,0,Ребенок,0,Ребенок,гнев,восторг,0.678856,1,0,NaN,NaN,Child,Child,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,3,Женщина,3,Женщина,разочарование,презрение,0.743977,1,0,Female,Female,NaN,NaN,0.0,NaN
8124,3,Женщина,3,Женщина,разочарование,агрессия,0.556850,1,0,Female,Female,NaN,NaN,0.0,NaN
8125,3,Женщина,3,Женщина,раскаяние,презрение,0.578382,1,0,Female,Female,NaN,NaN,0.0,NaN
8126,3,Женщина,3,Женщина,раскаяние,агрессия,0.478956,1,0,Female,Female,NaN,NaN,0.0,NaN


In [15]:
def comparable_groups(row):
    # Возвращаем True, если обе из возрастных групп (Child, Adult) или обе из гендерных (Male, Female)
    age_groups = ['Ребенок', 'Взрослый']
    gender_groups = ['Мужчина', 'Женщина']

    p1 = row['person1_name']
    p2 = row['person2_name']

    # Если обе в возрастных — ок
    if (p1 in age_groups) and (p2 in age_groups):
        return True
    # Если обе в гендерных — ок
    if (p1 in gender_groups) and (p2 in gender_groups):
        return True

    # Иначе — разные типы, не сравниваем
    return False

result_df_filtered = test_persons[test_persons.apply(comparable_groups, axis=1)].copy()

In [16]:
result_df_filtered

,person1_id,person1_name,person2_id,person2_name,emotion1,emotion2,cos,Is_diff_emotion,Is_diff_person,person1_gender,person2_gender,person1_age,person2_age,Is_diff_gender,Is_diff_age_group
0,0,Ребенок,0,Ребенок,гнев,изумление,0.559539,1,0,NaN,NaN,Child,Child,NaN,0.0
1,0,Ребенок,0,Ребенок,гнев,отвращение,0.480251,1,0,NaN,NaN,Child,Child,NaN,0.0
2,0,Ребенок,0,Ребенок,гнев,ужас,0.712524,1,0,NaN,NaN,Child,Child,NaN,0.0
3,0,Ребенок,0,Ребенок,гнев,восхищение,0.615569,1,0,NaN,NaN,Child,Child,NaN,0.0
4,0,Ребенок,0,Ребенок,гнев,восторг,0.678856,1,0,NaN,NaN,Child,Child,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,3,Женщина,3,Женщина,разочарование,презрение,0.743977,1,0,Female,Female,NaN,NaN,0.0,NaN
8124,3,Женщина,3,Женщина,разочарование,агрессия,0.556850,1,0,Female,Female,NaN,NaN,0.0,NaN
8125,3,Женщина,3,Женщина,раскаяние,презрение,0.578382,1,0,Female,Female,NaN,NaN,0.0,NaN
8126,3,Женщина,3,Женщина,раскаяние,агрессия,0.478956,1,0,Female,Female,NaN,NaN,0.0,NaN


In [18]:
result_df_filtered.to_csv('pairs.csv', index=False)